# Detecting Corona from Lung X-Rays

<div class="alert alert-danger">
*Disclaimer: Only assumed method to detect Corona from X-Ray Scans, Complete reliablity of this method is yet to be established. Main purpose of this notebook is to try out **albumentations***
</div>

<img src="https://user-images.githubusercontent.com/30664477/82361298-4a0da900-9a28-11ea-937d-81dbcd5a5be5.gif" width="400px">

In [1]:
%%capture
!pip install --upgrade efficientnet-pytorch

In [2]:
import torchvision
import torch
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
import torch.nn.functional as F
from albumentations import Compose, RandomCrop, Normalize, HorizontalFlip, Resize
from albumentations.pytorch import ToTensor


import random,os,sys
from tqdm.notebook import tqdm

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(1234)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_path = '/kaggle/input/covid-19-x-ray-10000-images/dataset/'

In [6]:
# transforms = Compose([
#     Resize(256, 256), 
#     RandomCrop(224, 224),
#     HorizontalFlip(),
#     Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225],
#     ),
#     ToTensor()
# ])

transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    ])

In [7]:
train_dataset = torchvision.datasets.ImageFolder(
        root=train_path,
        transform=transforms
    )

In [8]:
validation_split=0.2
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)

In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, 
                                           sampler=train_sampler,num_workers=0)
validation_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,
                                                sampler=valid_sampler,num_workers=0)

In [10]:
model = torchvision.models.resnet18()
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)
model = model.to(device)

In [11]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
num_epochs=5
dataloaders={
    'training':train_loader,
    'validation':validation_loader
}

dataset_sizes={
    'training':len(train_indices),
    'validation':len(val_indices)
}

In [13]:
with tqdm(total=num_epochs, file=sys.stdout, position=0, leave=True) as pbar:
    for epoch in range(num_epochs):
        for phase in ['training', 'validation']:
            if phase is 'training':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0.0
            with tqdm(total=len(dataloaders[phase]), position=0, leave=True) as batch_bar:
                for inputs, targets in dataloaders[phase]:
                    inputs = inputs.to(device)
                    targets = targets.to(device,dtype=torch.float)
                    targets=targets.view(-1, 1)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'training'):
                        outputs = model(inputs)
                        loss = criterion(outputs, targets)
                        _, predictions = torch.max(outputs, 1)

                    if phase is 'training':
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(predictions == targets.data)
                    batch_bar.update(1)
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]
                batch_bar.set_description('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase.title(), epoch_loss, epoch_acc))
        pbar.update(1)
        pbar.set_description('{}/{} Completed'.format(epoch+1, num_epochs))